# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [2]:
### General parameters
max_steps = 5e5 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 1000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "ShooterTraining" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 128 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [3]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'ML_Academy' started successfully!


Unity Academy name: ML_Academy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: ML_Brain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 6
        Action space type: continuous
        Action space size (per agent): 3
        Memory space size (per agent): 0
        Action descriptions: , , 


### Train the Agent(s)

In [4]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 1000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 2000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 3000. Mean Reward: 0.030952380952380953. Std of Reward: 0.127686549312304.
Step: 4000. Mean Reward: 0.015853658536585366. Std of Reward: 0.09268292682926829.
Step: 5000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 6000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 7000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 8000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 9000. Mean Reward: 0.016666666666666666. Std of Reward: 0.09172076325837246.
Step: 10000. Mean Reward: 0.014634146341463414. Std of Reward: 0.09255446810248916.
Step: 11000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 12000. Mean Reward: 0.015476190476190477. Std of Reward: 0.09160480287243158.
Step: 13000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 14000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 15000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 16000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 17000. Mean Reward: 0.0.

Step: 132000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 133000. Mean Reward: 0.015853658536585366. Std of Reward: 0.09268292682926829.
Step: 134000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 135000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 136000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 137000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 138000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 139000. Mean Reward: 0.015853658536585366. Std of Reward: 0.0926829268292683.
Step: 140000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 141000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 142000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 143000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 144000. Mean Reward: 0.015476190476190477. Std of Reward: 0.09160480287243158.
Step: 145000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 146000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 147000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 148000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 149000. Mean

Step: 267000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 268000. Mean Reward: 0.015853658536585366. Std of Reward: 0.0926829268292683.
Step: 269000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 270000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 271000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 272000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 273000. Mean Reward: 0.015476190476190477. Std of Reward: 0.09160480287243158.
Step: 274000. Mean Reward: 0.015853658536585366. Std of Reward: 0.0926829268292683.
Step: 275000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 276000. Mean Reward: 0.0011904761904761906. Std of Reward: 0.007622766949324821.
Step: 277000. Mean Reward: 0.014634146341463414. Std of Reward: 0.09255446810248916.
Step: 278000. Mean Reward: 0.015476190476190477. Std of Reward: 0.09160480287243158.
Step: 279000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 280000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 281000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 282000. Mean Rew

Saved Model
Step: 401000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 402000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 403000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 404000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 405000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 406000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 407000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 408000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 409000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 410000. Mean Reward: 0.015476190476190477. Std of Reward: 0.0916048028724316.
Step: 411000. Mean Reward: 0.015476190476190477. Std of Reward: 0.0916048028724316.
Step: 412000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 413000. Mean Reward: 0.0. Std of Reward: 0.0.
Step: 414000. Mean Reward: 0.015476190476190477. Std of Reward: 0.0916048028724316.
Step: 415000. Mean Reward: 0.0012195121951219512. Std of Reward: 0.007712872341874096.
Step: 416000. Mean Reward: 0.014285714285714285. Std of Reward: 0.09147320339

INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.


### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [5]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-500001.cptk


INFO:tensorflow:Froze 7 variables.


INFO:tensorflow:Froze 7 variables.


Converted 7 variables to const ops.
